In [1]:
import os
import sys
import numpy as np
from pprint import pprint
from tqdm import tqdm
import pandas as pd
from importlib import reload
import math
from IPython.display import display, Markdown

sys.path.insert(0, os.path.abspath('../'))
pd.options.display.max_rows = 999

import tesi_sabella.pyfluxc.pyfluxc as flux
import tesi_sabella.data_generator as generator


def mprint(s):
    display(Markdown(s))


reload(flux)
reload(generator)

<module 'tesi_sabella.data_generator' from '/Users/swedishhdcamera/Desktop/Projects/tesi_sabella/tesi_sabella/data_generator.py'>

# Generating time series

Create a InfluxDB Flux client and a data-generator that can poll InfluxDB to get new samples. Poll data added in the last 48 hours

In [8]:
fclient = flux.FluxClient(); 
fluxbucket = 'CIC_IDS_2017_Tuesday'
# start, end = fclient.bucket_timerange(fluxbucket)
cicids2017 = generator.ntop_Generator(fluxbucket, '15s', fclient, start)

Extract data from influxdb in time intervals due to kill signal

In [9]:
from datetime import datetime

pbar = tqdm()
lastpoll_start = start

startT=datetime.now()

iterations = math.ceil((end - start) / pd.Timedelta(minutes=60))
for i in tqdm(range(iterations)):
    lastpoll_end = lastpoll_start + pd.DateOffset(minutes=60)
    aus = cicids2017.poll(stop=lastpoll_end)
    lastpoll_start = aus if aus is not None else lastpoll_end
    break

stopT=datetime.now()
print(stopT-startT)    

df = cicids2017.toPandas()








0it [00:00, ?it/s]







  0%|          | 0/14 [00:00<?, ?it/s]

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

NaN values processing

In [4]:
# Fix score which is default NaN
df_clean = df.fillna({"score:score": 0})
# Dropping
nans_count = df_clean[df_clean.isnull().any(axis=1)]
print(f'NaNs count: {len(nans_count)}')
df_clean = df_clean.dropna()

NaNs count: 236


Generic information

In [5]:
devices = list(df_clean.index.droplevel(2).unique())
timeseries_lens = [len(df_clean.loc[dtype].loc[ip]) for dtype, ip in devices]
mean_len = np.mean(timeseries_lens)
min_len = np.min(timeseries_lens)

display(Markdown(f'#### Features count: {len(df_clean.columns)}'))
display(Markdown(f'#### Timeseries length: {int(mean_len)} (min. {min_len})'))
display(Markdown(f'#### Hosts:'))
pprint(devices)

#### Features count: 44

#### Timeseries length: 1068 (min. 1053)

#### Hosts:

[('pc', '192.168.10.12'),
 ('pc', '192.168.10.14'),
 ('pc', '192.168.10.15'),
 ('pc', '192.168.10.16'),
 ('pc', '192.168.10.17'),
 ('pc', '192.168.10.19'),
 ('pc', '192.168.10.25'),
 ('pc', '192.168.10.5'),
 ('pc', '192.168.10.8'),
 ('pc', '192.168.10.9'),
 ('server', '192.168.10.3'),
 ('server', '192.168.10.50'),
 ('server', '192.168.10.51')]


# Store time series

In [6]:
df_clean.to_pickle(f'../dataset/CIC_IDS_2017/{fluxbucket}.pkl')